<a href="https://colab.research.google.com/github/HYLee810/projectmanager/blob/main/%EA%B0%95%EC%82%AC_KLUE_BERT_%ED%86%A0%ED%81%B0_%EB%B6%84%EB%A5%98(%EC%98%88%EC%A0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [3]:
from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained(
    "klue/bert-base", num_labels=13, id2label={
    0: 'B-DT', 1: 'I-DT',  2: 'B-LC',  3: 'I-LC', 4: 'B-OG',
    5: 'I-OG',  6: 'B-PS',  7: 'I-PS', 8: 'B-QT', 9: 'I-QT',
    10: 'B-TI', 11: 'I-TI', 12: 'O'})
model.to('cuda:0')

Some weights of BertForTokenClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [4]:
from datasets import load_dataset
ner_train = load_dataset('klue', 'ner', split='train')
ner_val = load_dataset('klue', 'ner', split='validation')

Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
ner_train[0]

In [5]:
!git clone https://gist.github.com/220bdfde44d6fcdd78375877370a39f6.git preproc
!mv preproc/ner_preproc.py .

Cloning into 'preproc'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
Receiving objects: 100% (12/12), done.
Resolving deltas: 100% (3/3), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0


In [6]:
from tokenizers.pre_tokenizers import BertPreTokenizer
import ner_preproc

preproc = ner_preproc.TokenAlignPreprocessor(
    tokenizer=tokenizer,
    pre_tokenizer=BertPreTokenizer(),
    outside_label_id=12)  # [TODO] O 레이블의 번호

train_ds = ner_train.map(preproc.convert_example)
eval_ds = ner_val.map(preproc.convert_example)

Map:   0%|          | 0/21008 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [7]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    evaluation_strategy="epoch")

from transformers import DataCollatorForTokenClassification
data_collator = \
    DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.055600,0.064416


TrainOutput(global_step=2626, training_loss=0.0778787424794972, metrics={'train_runtime': 345.8907, 'train_samples_per_second': 60.736, 'train_steps_per_second': 7.592, 'total_flos': 572113064143440.0, 'train_loss': 0.0778787424794972, 'epoch': 1.0})

In [8]:
from transformers import pipeline
ner = pipeline(
    'token-classification',
    model=model,
    tokenizer=tokenizer,
    device='cuda:0')

In [9]:
ner('''오늘 12시 서울특별시 성북구에 있는 국민대 경영대학원에서
유재명 교수가 1가지 주제로 수업을 한다.''')

[{'entity': 'B-DT',
  'score': 0.9424917,
  'index': 1,
  'word': '오늘',
  'start': 0,
  'end': 2},
 {'entity': 'B-TI',
  'score': 0.9696398,
  'index': 2,
  'word': '12',
  'start': 3,
  'end': 5},
 {'entity': 'I-TI',
  'score': 0.9973273,
  'index': 3,
  'word': '##시',
  'start': 5,
  'end': 6},
 {'entity': 'B-LC',
  'score': 0.98305833,
  'index': 4,
  'word': '서울특별시',
  'start': 7,
  'end': 12},
 {'entity': 'I-LC',
  'score': 0.95102656,
  'index': 5,
  'word': '성북구',
  'start': 13,
  'end': 16},
 {'entity': 'B-OG',
  'score': 0.88725704,
  'index': 9,
  'word': '국민대',
  'start': 21,
  'end': 24},
 {'entity': 'I-OG',
  'score': 0.73936045,
  'index': 10,
  'word': '경영',
  'start': 25,
  'end': 27},
 {'entity': 'I-OG',
  'score': 0.696194,
  'index': 11,
  'word': '##대',
  'start': 27,
  'end': 28},
 {'entity': 'I-OG',
  'score': 0.67532676,
  'index': 12,
  'word': '##학',
  'start': 28,
  'end': 29},
 {'entity': 'I-OG',
  'score': 0.57003444,
  'index': 13,
  'word': '##원',
  'start